# 실험 1: AG News Full Text + BERT

이 섹션에서는 AG News 데이터셋의 **전체 뉴스 본문**을 기반으로 BERT 분류기를 학습합니다.

- `datasets` 라이브러리를 이용해 AG News 전체 데이터를 자동 다운로드합니다.
- `text` 컬럼의 전체 뉴스 본문을 그대로 사용하여 입력 문장을 구성합니다.
- short text 실험과 비교하기 위해 동일한 토크나이저 및 모델 구조를 사용합니다.

---

## 데이터 로딩 및 라이브러리 안내

이 노트북은 HuggingFace `datasets` 라이브러리를 통해 AG News 데이터를 자동으로 다운로드합니다.  
따라서 별도의 데이터 파일을 GitHub에 업로드하지 않아도 됩니다.

### 필요한 라이브러리:

- `datasets`
- `transformers`
- `torch`

아래 명령어로 설치 가능합니다:

```bash
pip install datasets transformers torch


In [1]:
# 필요 라이브러리 설치 (커널 환경에 확실히 반영됩니다)
%pip install --upgrade transformers datasets evaluate accelerate scikit-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import numpy as np
from datasets import load_dataset
from evaluate import load
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 데이터셋 로드
raw_datasets = load_dataset("ag_news")

# 토크나이저 로드
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# 토큰화 함수 정의
def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

# 토큰화 적용
tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)

# label 컬럼 이름 수정
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# 데이터셋 포맷 변경 (PyTorch 텐서로)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# 학습/평가 데이터셋 준비
train_ds = tokenized_datasets["train"]
eval_ds = tokenized_datasets["test"]

In [4]:
# 레이블 수 계산
num_labels = len(set(raw_datasets["train"]["label"]))

# 사전학습된 BERT 로드 + 출력층 설정
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_labels
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Accuracy metric 불러오기
metric = load("accuracy")

# Trainer에 넘길 compute_metrics 함수 정의
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return metric.compute(predictions=preds, references=labels)

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./baseline_bert",        # 모델과 체크포인트가 저장될 디렉토리
    do_train=True,                       # 학습 모드 켜기
    do_eval=True,                        # 평가 모드 켜기
    eval_steps=500,                      # 매 500 스텝마다 평가 수행
    save_steps=500,                      # 매 500 스텝마다 체크포인트 저장
    logging_steps=50,                    # 매 50 스텝마다 로그 출력
    learning_rate=2e-5,                  # 학습률
    per_device_train_batch_size=16,      # 학습 배치 크기
    per_device_eval_batch_size=32,       # 평가 배치 크기
    num_train_epochs=3,                  # 총 에폭 수
    weight_decay=0.01,                   # 가중치 감쇠 (L2 정규화)
    logging_dir="./logs",                # TensorBoard 로그 디렉토리
    fp16=True,                           # GPU FP16(mixed-precision) 사용
)

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    compute_metrics=compute_metrics
)
trainer.train()

Step,Training Loss
50,1.056700
100,0.470600
150,0.401800
200,0.359500
250,0.343500
300,0.352200
350,0.297200
400,0.316500
450,0.333200
500,0.280400


TrainOutput(global_step=22500, training_loss=0.151144024626414, metrics={'train_runtime': 2661.2741, 'train_samples_per_second': 135.274, 'train_steps_per_second': 8.455, 'total_flos': 4.736084041728e+16, 'train_loss': 0.151144024626414, 'epoch': 3.0})

In [9]:
# 8) 평가
results = trainer.evaluate()
print(f"Baseline Evaluation Results: {results}")

Baseline Evaluation Results: {'eval_loss': 0.23277249932289124, 'eval_accuracy': 0.9482894736842106, 'eval_runtime': 12.4335, 'eval_samples_per_second': 611.25, 'eval_steps_per_second': 19.142, 'epoch': 3.0}


# 실험 2: AG News Short Text (Headline Only) 실험

이 섹션에서는 AG News 데이터셋의 뉴스 본문이 아닌,
**헤드라인(첫 문장)만 추출한 short text**를 기반으로 BERT 분류기를 학습합니다.

- 전체 본문 대신 `text.split(".")[0]` 로 첫 문장만 사용
- short text 환경에서 BERT의 성능을 측정
- full text 결과와의 비교 분석을 위해 동일한 모델 구조 사용

In [ ]:
from datasets import load_dataset

# 데이터셋 로드
dataset = load_dataset("ag_news")

# 헤드라인 추정: 'text'에서 첫 문장만 추출
def extract_headline(text):
    return text.split(".")[0]  # 마침표 기준으로 첫 문장 추출

train_texts = [extract_headline(x['text']) for x in dataset['train']]
train_labels = [x['label'] for x in dataset['train']]

test_texts = [extract_headline(x['text']) for x in dataset['test']]
test_labels = [x['label'] for x in dataset['test']]

In [ ]:
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split

# BERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 토큰화 함수 정의
def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

# 토큰화 수행
train_encodings = tokenizer(train_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
test_encodings = tokenizer(test_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

In [ ]:
from torch.utils.data import Dataset

class TitleOnlyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=64):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=max_length,
            return_tensors="pt"
        )
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Dataset 생성
baseline_train_dataset = TitleOnlyDataset(train_texts, train_labels, tokenizer)
baseline_test_dataset = TitleOnlyDataset(test_texts, test_labels, tokenizer)

In [ ]:
import torch.nn as nn
from transformers import BertModel

# 기본 BERT 분류기 정의
class BaselineBERTClassifier(nn.Module):
    def __init__(self, pretrained_model_name='bert-base-uncased', num_classes=4):
        super(BaselineBERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.pooler_output  # [CLS] 벡터
        cls_output = self.dropout(cls_output)
        return self.classifier(cls_output)

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
from torch.utils.data import DataLoader

def train_baseline(model, dataset, epochs=3, batch_size=32, learning_rate=2e-5, device='cuda' if torch.cuda.is_available() else 'cpu'):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    loss_fn = CrossEntropyLoss()

    model.to(device)
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        print(f"\n[Epoch {epoch+1}/{epochs}]")

        for i, batch in enumerate(tqdm(dataloader, desc="Training")):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

            if i % 100 == 0:
                print(f"Batch {i}/{len(dataloader)} | Loss: {loss.item():.4f}")

        avg_loss = total_loss / len(dataloader)
        print(f"✅ Epoch {epoch+1} Completed | Avg Loss: {avg_loss:.4f}")

In [ ]:
# 모델 인스턴스화 및 학습 실행
baseline_model = BaselineBERTClassifier(num_classes=4)

train_baseline(
    model=baseline_model,
    dataset=baseline_train_dataset,
    epochs=3,
    batch_size=32,
    learning_rate=2e-5
)

In [ ]:
def evaluate_baseline_model(model, dataset, batch_size=32, device='cuda' if torch.cuda.is_available() else 'cpu'):
    dataloader = DataLoader(dataset, batch_size=batch_size)
    model.to(device)
    model.eval()

    correct = 0
    total = 0
    total_loss = 0
    loss_fn = CrossEntropyLoss()

    from time import time
    start_time = time()

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating Baseline"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            preds = torch.argmax(outputs, dim=1)

            total_loss += loss.item()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    end_time = time()
    acc = correct / total
    avg_loss = total_loss / len(dataloader)
    runtime = end_time - start_time

    result = {
        'eval_loss': avg_loss,
        'eval_accuracy': acc,
        'eval_runtime': round(runtime, 4),
        'eval_samples_per_second': round(total / runtime, 2),
        'eval_steps_per_second': round(len(dataloader) / runtime, 2)
    }

    print(f"✅ Baseline Evaluation Results: {result}")
    return result

In [ ]:
# 평가 실행
baseline_eval_results = evaluate_baseline_model(baseline_model, baseline_test_dataset)